In [83]:
%load_ext autoreload
%autoreload 2

import os

if 'cachai' not in os.listdir('.'):
    os.chdir('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
from enum import Enum, auto


class ObservationType(Enum):
    HIT = auto()
    MISS = auto()
    STALE = auto()
    VALID_TTL = auto()

    def __str__(self):
        return self.name

In [85]:
import numpy as np
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_error


class TTLSimulator:

    def __init__(self, iterations=1_000):
        self._iterations = iterations
        self._target_params = [
            (50, 5),
            (200, 10),
            (400, 30),
        ]
        # means = np.linspace(10, 500, 10).astype(int)
        # std = np.arange(1, len(means) + 1)**2
        # self._target_params = np.array([means, std]).T

    def generate_features(self, target, num_features=1, correlation=0.8):
        cov_matrix = np.eye(num_features) * (1 - correlation) + np.ones((num_features, num_features)) * correlation
        features = np.random.multivariate_normal(np.ones(num_features) * target, cov_matrix)
        return features.reshape(1, -1)
        # return np.full((1, num_features), fill_value=target)

    def update_target_params(self, progress):
        target_params = []
        for param in self._target_params:
            mean = float(
                round(param[0]*np.sin(progress*2*np.pi/2)/(param[0]/2) + param[0], 2)
            )
            std = param[1]
            target_params.append((mean, std))
        self._target_params = target_params

    def feedback(self, y_pred, y):
        observation_time = int(min(y_pred, y)[0])
        hits = max(0, observation_time-1)
        observation_type = None
        if y_pred[0] < y[0]:
            observation_type = ObservationType.MISS
        elif y_pred[0] > y[0]:
            observation_type = ObservationType.STALE
        else:
            observation_type = ObservationType.VALID_TTL
        return observation_time, observation_type, hits

    def generate(self):
        target_param_index = np.random.randint(0, len(self._target_params))
        target_params = self._target_params[target_param_index]
        y = np.random.normal(target_params[0], target_params[1], 1)
        X = self.generate_features(y)
        return X, y

In [86]:
class Experiment():

    def __init__(self, simulator, iterations=1_000):
        self._simulator = simulator
        self._iterations = iterations

    def score(self, y_pred, y):
        mse = root_mean_squared_error(y_pred, y)
        mae = mean_absolute_error(y_pred, y)
        return mse, mae

    def run(self, model):
        history = []
        for i in range(self._iterations):
            X, y = self._simulator.generate()
            y_pred = model.predict(X)
            observation_time, observation_type, hits = self._simulator.feedback(y_pred, y)
            model.observe(observation_time, observation_type, hits, y_pred)
            scores = self.score(y_pred, y)
            history.append([i, observation_type, observation_time, y_pred[0], y[0], hits, *scores])
        df = pd.DataFrame(history, columns=[
            'iteration', 'observation_type', 'observation_time',
            'y_pred', 'y', 'hits', 'mse', 'mae'
        ])
        return df

In [87]:
from abc import ABC, abstractmethod


class BaseModel(ABC):

    @abstractmethod
    def predict(self, X: np.array) -> np.array:
        pass

    @abstractmethod
    def observe(
        self,
        observation_time: int,
        observation_type: ObservationType,
        hits: int,
        prev_prediction: float
    ) -> None:
        pass

In [88]:
class Model(BaseModel):
    def __init__(self):
        pass

    def predict(self, X):
        output = X.mean()
        return np.array([output])

    def observe(self, observation_time, observation_type, hits, prev_prediction):
        a = 2+2

In [89]:
simulator = TTLSimulator()
model = Model()
experiment = Experiment(simulator)

df = experiment.run(model)
df

,iteration,observation_type,observation_time,y_pred,y,hits,mse,mae
0,0,MISS,176,176.853127,177.568088,175,0.714961,0.714961
1,1,MISS,54,54.636339,55.539773,53,0.903434,0.903434
2,2,STALE,55,57.056920,55.927715,54,1.129205,1.129205
3,3,STALE,202,203.155134,202.558422,201,0.596712,0.596712
4,4,STALE,55,55.763187,55.386399,54,0.376787,0.376787
...,...,...,...,...,...,...,...,...
995,995,STALE,184,185.056245,184.026096,183,1.030149,1.030149
996,996,MISS,48,48.396355,49.699637,47,1.303282,1.303282
997,997,MISS,46,46.868692,46.884552,45,0.015860,0.015860
998,998,MISS,205,205.409917,206.185309,204,0.775392,0.775392
